# Symboltabellen

In diesem Notebook schauen wir uns Symboltabellen an. Wir diskutieren eine sehr einfache Implementation, die auf einem geordneten Array basiert.

Bevor wir aber zur Implementation kommen, machen wir noch einen kleinen Exkurs zum Thema Objektgleichheit in Python.

### Objektgleichheit

Die wichtigste Operation in Symboltabellen ist das Suchen von zu einem Schlüssel dazugehörenden Wert. Diese Operation liefert den Wert in der Symboltabelle zurück, der unter dem Schlüssel in der Symboltabelle gespeichert ist, der *gleich* dem Anfrageschlüssel ist. Die *Gleichheit* von Schlüsseln spielt hier also eine wichtige Rolle. Deshalb werden wir uns, bevor wir zu den Implementationen kommen, zuerst Objektgleichheit in Python genauer anschauen. 

Intuitiv ist uns klar, was es bedeutet, dass zwei Objekte gleich sind. Zum Beispiel wissen wir, dass wenn

In [ ]:
s1 = "Hallo"
s2 = "Hallo"

dann sollte auch s1 und s2 gleich sein. Also möchten wir, dass folgender Ausdruck ```true``` ergibt:

In [ ]:
s1 == s2

Für die von Python vordefinierten Typen wie Strings, Int, Float, etc. stimmt dies auch tatsächlich. Wenn wir aber unsere eigenen Typen definieren, wird die Sache etwas schwieriger. Wir veranschaulichen dies am Beispiel der Klasse Datum, die wir wie folgt definieren:

In [ ]:
class Date:
    def __init__(self, day, month, year):
        self.day = day
        self.month = month
        self.year = year

Jetzt beobachten wir, was passiert, wenn wir zwei Objekte vom Typ Datum vergleichen:

In [ ]:
d1 = Date(1, 1, 2018)
d2 = Date(1, 1, 2018)
d1 == d2

Wir sehen, dass die Objekte ```d1``` und ```d2``` von Python als ungleich erachtet werden. Der Grund ist, dass per default die Gleichheit nur auf Objektidentitäten definiert ist. Das bedeutet, es wird nur verglichen, ob es sich um das gleiche Objekt handelt, aber nicht, ob der Inhalt gleich ist. Wenn wir Objekte als Schlüssel für Symboltabellen verwenden wollen, reicht diese Definition von Gleichheit nicht aus.

Damit wir auch für benutzerdefinierte Typen richtige Objektgleichheit haben, müssen wir die Methode ```__eq__``` implementieren. Diese Methode muss eine Äquivalenzrelation definieren, also folgende Anforderungen erfüllen:

* Reflexivität: $x == x$ ist wahr
* Symmetrie: $x == y$ genau dann, wenn $y == x$
* Transitivität: $x ==y $ und $y == z$ dann $x == z$

In der Praxis ist die Implementation der Methode ```__eq__``` meist sehr einfach, da wir diese direkt über die Felder der Klasse definieren können. 

In [ ]:
class Date:
    def __init__(self, day, month, year):
        self.day = day
        self.month = month
        self.year = year
        
    def __eq__(self, rhs):
        return self.day == rhs.day \
            and self.month == rhs.month \
            and self.year == rhs.year

Wenn wir jetzt nochmals zwei Objekte vom Typ ```Date``` definieren, sollte alles richtig funktionieren:

In [ ]:
d1 = Date(1, 1, 2018)
d2 = Date(1, 1, 2018)
d3 = Date (2, 2, 2018)
print("d1==d2: ", d1 == d2)
print("d1==d3: ", d1 == d3)
print("d1!=d3: ", d1 != d3)


## Implementation 1: Implementation mit geordnetem Array

Wir kommen nun zur ersten einfachen Implementation einer Symboltabelle. Das Herzstück dieser Implementation ist die Methode ```rank```, welche, gegeben einen Schlüssel und ein geordnetes Array, die Anzahl Elemente im Array zurückgibt, die kleiner als der Schlüssel sind. 

In [ ]:
def rank(a , key ):
    lo = 0
    hi = len(a) - 1
    while lo  <= hi:
        mid = (lo + hi) // 2        
        if  a[mid] < key:
            lo = mid + 1
        elif  key  < a[mid]:
            hi = mid - 1
        else:
            return mid
    return lo

#### Übung:

* Machen Sie eine erfolgreiche und erfolglose Suchanfrage und vergewissern Sie sich, dass der Rückgabewert genau der Anzahl Elementen entspricht, die kleiner als der gesuchte Schlüssel sind. 

Die Implementation der Symboltabelle ist nun  einfach. Die Idee ist, dass wir die Schlüssel in einem sortierten Array speichern. Die Werte werden in einem separaten Array an der entsprechenden Position gespeichert. Dank der Methode ```rank``` können wir effizient die Stelle im Array finden, in der wir den Schlüssel erwarten oder wo wir den neuen Schlüssel einfügen müssen.

In [1]:
class BinarySearchST:
    
    def __init__(self):
        self._keys = []
        self._values = []
    
    
    def _rank(self, value):
        lo = 0
        hi = len(self._keys) - 1
        while lo  <= hi:
            mid = lo + (hi - lo) // 2
            if  self._keys[mid] < value:
                lo = mid + 1
            elif  value  < self._keys[mid]:
                hi = mid - 1
            else:
                return mid
        return lo
    
    def get(self, key):
        if self.isEmpty():
            return None
        
        rank = self._rank(key)
        if rank < len(self._keys) and self._keys[rank] == key:
            return self._values[rank]
        else:
            return None
        
    def put(self, key, value):
        rank = self._rank(key)
        if rank < len(self._keys) and self._keys[rank] == key:
            self._values[rank] = value
        else:
            self._keys.insert(rank, key)
            self._values.insert(rank, value)
             
    def delete(self, key):
        rank = self._rank(key)
        if rank < len(self._keys) and self._keys[rank] == key:
            del self._keys[rank] 
            del self._values[rank]
            
    
    def contains(self, key):
        rank = self._rank(key)
        return rank < len(self._keys) and self._keys[rank] == key
    
    def isEmpty(self):
        return self.size() == 0
    
    def size(self):
        return len(self._keys)
    
    def keys(self):
        return self._keys

#### Übung:
* Schauen Sie sich die Implementation genau an. Wie wird ```rank``` verwendet? Was muss jeweils zusätzlich noch gemacht werden?



### Clients

Im Folgenden zeigen wir die Nutzung unserer Symboltabelle an zwei einfachen Beispielen:

#### Testclient

Unser erster Client nimmt jeden Buchstaben von einem String als Key und die Position des Buchstabens im String als Wert.

In [2]:
string = "SEARCHEXAMPLE"
st = BinarySearchST()
for (pos, char) in enumerate(string):
    st.put(char, pos)

Um alle Objekte auszugeben, gehen wir durch alle Schlüssel und geben die dazugehörigen Werte aus:

In [3]:
for k in st.keys():
    print("key: " +str(k) + " Wert: " +str(st.get(k)))

key: A Wert: 8
key: C Wert: 4
key: E Wert: 12
key: H Wert: 5
key: L Wert: 11
key: M Wert: 9
key: P Wert: 10
key: R Wert: 3
key: S Wert: 0
key: X Wert: 7


##### Übung:

* Ersetzen Sie einen bereits vorhandenen Schlüssel mit der Methode ```put```. Was passiert?

#### Frequency Client

Unser zweiter Client zeigt eine erste sinnvolle Anwendung von Symboltabellen. Er zählt, wie oft ein bestimmes Wort in einem Text vorkommt.

Als Erstes bereiten wir einen Text auf. Wir nutzen den folgenden Ausschnitt aus "Alice's Adventures in Wonderland". Damit wir nicht durch die Satzzeichen gestört werden, entfernen wir diese zuerst. 

In [4]:
testdataRaw = """ Alice was beginning to get very tired of 
sitting by her sister on the bank, and of having nothing to 
do:  once or twice she had peeped into the book her sister 
was reading, but it had no pictures or conversations in it, `and what is the use of a book,'
thought Alice `without pictures or conversation?'

  So she was considering in her own mind (as well as she could,
for the hot day made her feel very sleepy and stupid), whether
the pleasure of making a daisy-chain would be worth the trouble
of getting up and picking the daisies, when suddenly a White
Rabbit with pink eyes ran close by her.

  There was nothing so VERY remarkable in that; nor did Alice
think it so VERY much out of the way to hear the Rabbit say to
itself, `Oh dear!  Oh dear!  I shall be late!'  (when she thought
it over afterwards, it occurred to her that she ought to have
wondered at this, but at the time it all seemed quite natural);
but when the Rabbit actually TOOK A WATCH OUT OF ITS WAISTCOAT-
POCKET, and looked at it, and then hurried on, Alice started to
her feet, for it flashed across her mind that she had never
before seen a rabbit with either a waistcoat-pocket, or a watch to
take out of it, and burning with curiosity, she ran across the
field after it, and fortunately was just in time to see it pop
down a large rabbit-hole under the hedge.
"""
import string
translator = str.maketrans('', '', string.punctuation)
testdata = testdataRaw.translate(translator).lower()

##### Übung: 

* Was macht die Python Methode translate? Explorieren Sie.


Nun iterieren wir durch alle Wörter und nutzen das Wort jeweils als Schlüssel. Für jeden Schlüssel speichern wir dann, wie oft wir diesen Schlüssel (also das Wort) bereits im Text gesehen haben. 

In [5]:
st = BinarySearchST()
for word in testdata.split():
    if len(word) > 5:
        if st.contains(word):
            st.put(word, st.get(word) + 1)
        else:
            st.put(word, 1)

Wir können nun das Wort, welches am meisten vorkommt, ganz einfach auslesen. Beachten Sie, dass wir uns dazu eine Variable max führen, die immer auf das Wort mit den meisten Einträgen zeigt. Am Anfang wird dies auf ein nicht existierendes Wort, mit 0 Einträgen gesetzt.

In [6]:
maximum = ""
st.put("", 0)
for word in st.keys():
    if st.get(word) > st.get(maximum):
        maximum = word
print(maximum, st.get(maximum))

rabbit 4


#### Übung:
* Geben Sie alle im Text vorkommende Wörter, zusammen mit deren Häufigkeit aus

In [8]:
# Ihre Lösung